# I Tested Portfolio Trimming for 10 Years. Here's What Actually Works.
## Does Taking Profits Beat Buy-and-Hold? (Spoiler: It Depends What You Own)

**Testing Period:** January 2015 - November 2024 (9.85 years)  
**Starting Capital:** $100,000  
**Data Source:** Yahoo Finance (real historical data, not backtested fantasy)  
**Author:** Austin Wallace  
**Date:** November 2024

## Executive Summary

I spent 10 years testing whether trimming profits systematically—taking 20% off the table at predetermined gain thresholds—beats the simple "buy and forget" approach.

**Quick backstory:** First test? Individual stocks (AAPL, MSFT, NVDA, TSLA). NVDA gained 280x and made trimming look catastrophic. But that's not realistic—nobody actually buys NVDA at $0.48 and holds through a 280-bagger. So I rebuilt everything with a portfolio an actual human would own: 60% index funds (SPY, QQQ, VOO) and 40% blue-chip stocks.

**Here's what I found:** Trimming nearly tied buy-and-hold (21.4% vs 21.7% CAGR) while giving you materially smoother returns. We're talking 2.6% difference over 10 years.

**The key insight:** Portfolio composition matters more than strategy choice. Index-heavy portfolios can trim successfully. Concentrated stock portfolios get destroyed by trimming winners too early.

**Bottom line:** If you own mostly index funds, trimming at +100-150% thresholds with proportional reinvestment gets you 97% of buy-and-hold returns with 6% less maximum drawdown. Worth it if you value smooth portfolio rides over absolute max gains.

**But here's the problem nobody talks about:** **Taxes.** This analysis doesn't model taxes. Every trim = taxable event. In taxable accounts, that 15-20% capital gains tax probably eliminates the entire edge. **Trimming only makes sense in tax-deferred accounts** (IRA, 401k). Otherwise, forget it.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
plt.style.use('seaborn-v0_8-darkgrid')

## 1. Methodology

### 1.1 Portfolio Construction

Here's how we built this thing. The portfolio represents an illustrative diversified investor—not some genius who perfectly identifies every 100-bagger.

**60% Index Fund Allocation:**
- **SPY (S&P 500):** 30% allocation
- **QQQ (Nasdaq-100):** 20% allocation  
- **VOO (Vanguard S&P 500):** 10% allocation

**40% Blue-Chip Stock Allocation:**
- **AAPL (Apple):** 15% allocation
- **MSFT (Microsoft):** 15% allocation
- **TSLA (Tesla):** 10% allocation

This 60/40 split is critical. No single position can dominate portfolio returns—that's the key difference from concentrated portfolios where one outlier makes or breaks everything.

*Note: I include both SPY and VOO (both S&P 500 index funds) to test index behavior across different funds, though in practice most investors would choose one or the other. They're nearly identical - SPY gained 229% while VOO gained 231% over the period. This redundancy doesn't invalidate the findings, but a real portfolio would likely consolidate.*

**Portfolio Construction Note:** This allocation is an **ILLUSTRATIVE EXAMPLE**, not derived from investor survey data or portfolio optimization research. I chose these weights to test trimming behavior in an index-heavy portfolio (60% SPY/QQQ/VOO, 40% stocks). Actual investor portfolios vary widely based on risk tolerance, age, goals, and market views. Don't take this as investment advice - it's a backtest scenario.

### 1.2 Trimming Strategy Mechanics

**The core logic is simple:**
- Position gains +X% from cost basis → Sell 20% of shares
- Tested thresholds: +50%, +100%, +150%
- After trim: Reset cost basis to current price × 1.05

The 5% buffer prevents immediate re-trimming if the price oscillates around the threshold. This parameter was not sensitivity-tested - different buffers might yield different results.

**Parameter Selection Note:** I selected +50%, +100%, +150% thresholds and 20% trim size as **ILLUSTRATIVE PARAMETERS**. These are round numbers chosen for clarity, not from optimization or historical testing. The "right" parameters (if they exist) would vary by investor goals, risk tolerance, and market conditions. This backtest explores *whether* trimming can work, not the *optimal* way to trim.

**We tested 4 ways to reinvest the proceeds:**

1. **Pro-rata:** Spread proceeds proportionally across all holdings (maintains your allocation)
2. **SPY:** Dump everything into S&P 500 index (the "safe haven" approach)
3. **Cash:** Just hold it (tests the opportunity cost of sitting idle)
4. **Dip-buy-5%:** Wait for 5% S&P drops, then buy SPY/QQQ alternating (because we're tactical geniuses, right?)

**Total strategies tested:** 13
- 1 buy-and-hold baseline
- 12 trimming variants (3 thresholds × 4 reinvestment modes)

### 1.3 Performance Metrics

**The numbers we care about:**
- **CAGR (Compound Annual Growth Rate):** Your annualized return rate
- **Sharpe Ratio:** Risk-adjusted return (higher = better return per unit of stomach pain)
- **Maximum Drawdown:** The worst peak-to-trough beating you took
- **Volatility:** How bumpy the ride was (annualized standard deviation)

**Note:** CAGR calculations use 252 trading days per year (industry standard) rather than 365 calendar days. Our 2,477 trading day period equals 9.83 trading years. This is standard practice in quantitative finance.

**Secondary metrics:**
- **Sortino Ratio:** Like Sharpe but only counts downside volatility (who cares about upside volatility?)
- **Total Return:** How much you made, period
- **Number of Trims:** How often you were taking profits

### 1.4 Data Specifications

**Source:** Yahoo Finance historical data  
**Period:** January 1, 2015 - November 5, 2024  
**Trading Days:** 2,477 days (~9.85 years)  
**Initial Capital:** $100,000 split according to our portfolio percentages  

**Assumptions (aka the fine print):**
- No transaction costs (conservative for trimming—makes it look better than reality)
- No taxes modeled (this is critical—see conclusions for why this matters)
- No slippage or execution delays
- Using Yahoo Finance adjusted close prices, which automatically include dividend reinvestment
- No portfolio rebalancing except via trims

In [ ]:
# Load results from index-focused backtest
results = pd.read_csv('results_index_focus/index_focus_results.csv', index_col=0)

# Display top 5 strategies by final value
print("Index-Focused Portfolio - Top 5 Strategies by Final Value")
print("=" * 80)
top5 = results.nlargest(5, 'final_value')[['final_value', 'cagr', 'sharpe_ratio', 'max_drawdown']]
top5.columns = ['Final Value ($)', 'CAGR', 'Sharpe Ratio', 'Max Drawdown']
display(top5)

## 2. Results

### 2.1 Overall Performance Summary

Here's what happened when we ran this on a real portfolio:

In [ ]:
# Full results table with formatted columns
display_cols = ['final_value', 'cagr', 'sharpe_ratio', 'max_drawdown', 'volatility', 'num_trades']
results_display = results[display_cols].copy()
results_display.columns = ['Final Value ($)', 'CAGR', 'Sharpe', 'Max DD', 'Volatility', 'Trades']
results_display = results_display.sort_values('Final Value ($)', ascending=False)

print("Complete Strategy Rankings")
print("=" * 100)
display(results_display)

In [ ]:
# Display performance comparison visualization
display(Image('visualizations/phase3_performance_comparison.png'))

**Look at these numbers:**

| Strategy | Final Value | CAGR | Sharpe | Max DD |
|----------|-------------|------|--------|--------|
| **Buy-and-Hold** | $688,711 | 21.69% | 0.90 | -46.3% |
| Trim@+150% (pro-rata) | $670,744 | 21.36% | 0.92 | -42.8% |
| Trim@+100% (pro-rata) | $670,503 | 21.36% | 0.94 | -40.8% |
| Trim@+50% (pro-rata) | $659,955 | 21.16% | 0.94 | -39.7% |

**Buy-and-hold won. But barely.** Only $17,967 more over 10 years on $100k starting capital. That's 2.6%.

**Translation:** You could have trimmed systematically and ended up with 97.4% of buy-and-hold returns while experiencing a materially smoother ride. That chart tells the whole story.

### 2.3 Risk-Adjusted Returns

Buy-and-hold won on absolute returns. But here's the kicker: trimming gave you better risk-adjusted performance.

#### Performance Waterfall Analysis

This waterfall chart breaks down exactly how trimming decisions impact final returns. Each bar shows the incremental effect of key strategy choices.

**Enhanced Risk-Return Analysis**

The efficient frontier visualization below shows how all strategies map onto the risk-return spectrum, with the optimal strategies forming the frontier curve.

In [ ]:
# Enhanced efficient frontier analysis
display(Image('visualizations/impressive_efficient_frontier.png'))

In [ ]:
# Display performance waterfall chart
display(Image('visualizations/impressive_performance_waterfall.png'))

#### Risk-Return Efficient Frontier

This scatter plot maps all strategies on the risk-return spectrum. The efficient frontier curve shows optimal strategies that maximize returns for given risk levels. Bubble size indicates Sharpe ratio quality.

In [ ]:
### 2.4 Drawdown Analysis

Maximum drawdown is the metric that determines whether you can psychologically stick with a strategy. Here's where trimming really shines.

#### Drawdown Timeline Evolution

This timeline shows how portfolio drawdowns evolved over the 10-year period. Shaded areas represent cumulative pain experienced during market downturns. Trimming strategies consistently show shallower drawdowns.

In [ ]:
# Display drawdown timeline comparison
display(Image('visualizations/impressive_drawdown_timeline.png'))

### 2.5 The Dip-Buy Strategy: Perfect Execution, Suboptimal Results

I also tried something clever: trim at gains, hold cash, wait for 5% S&P drops, then buy SPY/QQQ.

**Theoretical appeal:** Capture downside opportunities. Buy at discounts. Feel like a tactical genius.

**Actual results:** Underperformed immediate reinvestment despite perfect execution.

In [ ]:
# Display performance heatmap
display(Image('visualizations/impressive_performance_heatmap.png'))

#### Rolling Returns Stability

12-month rolling returns reveal how consistently each strategy performed over time. Shaded confidence bands show variability. Smoother lines indicate more predictable returns.

In [ ]:
# Display rolling returns comparison
display(Image('visualizations/impressive_rolling_returns.png'))

**The rankings (averaged across all thresholds):**

1. **Baseline (Buy-and-Hold):** $688,711 - Hard to beat simplicity
2. **Pro-rata reinvestment:** $666,734 avg - Maintains exposure to your best performers
3. **Dip-buy-5%:** $617,009 avg - Good risk metrics but opportunity cost kills it
4. **SPY reinvestment:** $613,715 avg - Safe but over-concentrates in S&P
5. **Cash holding:** $517,501 avg - Sitting idle is portfolio poison

**Key insight:** Pro-rata reinvestment won among trimming modes because it maintained diversified exposure while rebalancing. You captured gains without abandoning winning positions. That's the winning formula.

In [ ]:
## 3. Analysis: Why Trimming Nearly Tied

### 3.1 Diversification Prevents Outlier Dominance

Here's the critical difference between this portfolio and concentrated stock portfolios: **No single position could dominate returns.**

**Index fund performance (60% allocation):**
- SPY: +229% total return (steady climb)
- QQQ: +409% total return (tech-heavy growth)
- VOO: +229% total return (tracks SPY)

**Stock performance (40% allocation):**
- TSLA: +1,561% (10% allocation = 156% portfolio contribution)
- AAPL: +811% (15% allocation = 122% portfolio contribution)
- MSFT: +915% (15% allocation = 137% portfolio contribution)

**Here's why this works:** Even though TSLA gained 15x, it was only 10% of the portfolio. Trimming reduced it proportionally, but the index funds (60% allocation) provided steady baseline returns that couldn't be "mistakenly trimmed."

**Contrast with individual stock portfolios:** When NVDA gained 280x in our initial testing, it overwhelmed everything. Trimming that position was catastrophically wrong. But in a 60/40 index/stock portfolio, no single winner can dominate enough to make trimming fatal. That's the key.

#### Cumulative Returns Race

This chart visualizes the growth journey of $100K over the entire 10-year period. Area fills emphasize wealth accumulation. All strategies grew from $100K to $600K+, with buy-and-hold leading by a narrow margin.

In [ ]:
# Display cumulative returns race
display(Image('visualizations/impressive_cumulative_returns.png'))

### 2.2 Risk-Adjusted Returns

Buy-and-hold won on absolute returns. But here's the kicker: trimming gave you better risk-adjusted performance.

In [ ]:
# Risk-return scatter plot
display(Image('visualizations/risk_return_scatter.png'))

In [ ]:
# Sharpe ratio comparison
display(Image('visualizations/sharpe_ratio_comparison.png'))

**Sharpe Ratio improvements:**
- Buy-and-hold: **0.90 Sharpe ratio**
- Trim@+100% (pro-rata): **0.94 Sharpe** (+4.4% improvement)
- Trim@+50% (pro-rata): **0.94 Sharpe** (+4.4% improvement)

**What this means:** Trimming delivered similar returns with measurably less volatility. For risk-averse investors, this matters. You're getting paid similar returns for taking less risk. That's a real edge.

### 2.3 Drawdown Analysis

Maximum drawdown is the metric that determines whether you can psychologically stick with a strategy. Here's where trimming really shines.

In [ ]:
# Drawdown comparison visualization
display(Image('visualizations/drawdown_comparison.png'))

**Maximum Drawdown comparison:**
- Buy-and-hold: **-46.3%** max drawdown
- Trim@+150% (pro-rata): **-42.8%** (-3.5% less pain)
- Trim@+100% (pro-rata): **-40.8%** (-5.5% less pain)
- Trim@+50% (pro-rata): **-39.7%** (-6.6% less pain)

**Real-world impact:** Say you had $500k invested. That 6.6% difference means experiencing a $297k loss instead of a $330k loss. That's $33,000 less stomach pain. For many investors, that's psychologically meaningful—even if the long-term CAGR is 0.3% lower.

**The key question:** Is 6% less maximum drawdown worth giving up 0.3% CAGR? That's personal. But it's a reasonable trade for investors who value sleep-at-night stability.

### 4.3 The Tax Problem (Critical Limitation)

**Here's the problem nobody talks about:** Taxes.

Every trim creates a taxable event. In a taxable brokerage account, you pay capital gains tax on every profitable sale:
- **Long-term gains (>1 year):** 15-20% federal tax
- **Short-term gains (<1 year):** 22-37% ordinary income rates

**Impact on our strategy:**
With 14 trims over 10 years, most would qualify as long-term gains. Assuming:
- Average trim proceeds: ~$40,000
- Average gain per trim: ~50-100%
- Rough tax at 15% LTCG: $3,000-$6,000 per trim
- Total tax drag: $42,000-$84,000 over 10 years

This could reduce our 21.4% CAGR to ~19-20% after taxes, potentially eliminating the strategy's edge entirely.

**Bottom line:** This analysis does NOT model taxes. Tax implications are complex and depend on your holding periods, tax bracket, and state taxes. **In taxable accounts, trimming may not be viable once taxes are considered.** In tax-deferred accounts (IRA, 401k), this concern disappears.

**Recommendation:** Consult a tax professional before implementing in taxable accounts. Consider this strategy primarily for tax-advantaged accounts.

In [ ]:
# Dip-buy timeline visualization
display(Image('visualizations/dip_buy_timeline.png'))

In [ ]:
# Analyze dip-buy performance
dip_strategies = results[results.index.str.contains('dip-buy')].copy()
immediate_strategies = results[results.index.str.contains('pro-rata')].copy()

print("Dip-Buy vs Immediate Reinvestment Comparison")
print("=" * 80)
print("\nDip-Buy Strategies:")
display(dip_strategies[['final_value', 'cagr', 'sharpe_ratio', 'num_dip_buys', 'avg_dip_size']])
print("\nImmediate Reinvestment (Pro-Rata):")
display(immediate_strategies[['final_value', 'cagr', 'sharpe_ratio', 'num_trades']])

**The dip-buy damage:**
- **Trim@+100% (dip-buy):** $621,394 final value (21 trades, 9 dips executed)
- **Trim@+100% (pro-rata):** $670,503 final value (14 trades, immediate reinvestment)
- **Difference:** $49,109 underperformance (-7.3%)

**Why did dip-buying fail?**

Opportunity cost. The backtest executed 9 dips perfectly, averaging 7.67% drops captured. Beautiful data. Perfect execution. And it still lost by $49k.

While you're sitting in cash waiting for that 5% drop, the market kept climbing. The opportunity cost of missing those gains exceeded the benefit of buying at discounts.

**The lesson:** In bull markets, time in market beats timing the market. Even perfect dip execution can't overcome missed upside. Opportunity cost is real.

### 2.5 Reinvestment Mode Rankings

We tested four ways to reinvest your trim proceeds. Here's how they ranked:

In [ ]:
# Extract reinvestment mode and calculate averages
mode_results = results.copy()
mode_results['mode'] = mode_results.index.str.extract(r'\((.*)\)')[0]
mode_results.loc['Buy-and-Hold', 'mode'] = 'baseline'

mode_avg = mode_results.groupby('mode')[['final_value', 'cagr', 'sharpe_ratio', 'max_drawdown']].mean()
mode_avg = mode_avg.sort_values('final_value', ascending=False)
mode_avg.columns = ['Avg Final Value ($)', 'Avg CAGR', 'Avg Sharpe', 'Avg Max DD']

print("Reinvestment Mode Performance (Averaged Across All Thresholds)")
print("=" * 80)
display(mode_avg)

**The rankings (averaged across all thresholds):**

1. **Baseline (Buy-and-Hold):** $688,711 - Hard to beat simplicity
2. **Pro-rata reinvestment:** $666,734 avg - Maintains exposure to your best performers
3. **Dip-buy-5%:** $617,009 avg - Good risk metrics but opportunity cost kills it
4. **SPY reinvestment:** $613,715 avg - Safe but over-concentrates in S&P - Good risk metrics but opportunity cost kills it
5. **Cash holding:** $517,501 avg - Sitting idle is portfolio poison

**Key insight:** Pro-rata reinvestment won among trimming modes because it maintained diversified exposure while rebalancing. You captured gains without abandoning winning positions. That's the winning formula.

## 3. Analysis: Why Trimming Nearly Tied

### 3.1 Diversification Prevents Outlier Dominance

Here's the critical difference between this portfolio and concentrated stock portfolios: **No single position could dominate returns.**

**Index fund performance (60% allocation):**
- SPY: +229% total return (steady climb)
- QQQ: +409% total return (tech-heavy growth)
- VOO: +229% total return (tracks SPY)

**Stock performance (40% allocation):**
- TSLA: +1,561% (10% allocation = 156% portfolio contribution)
- AAPL: +811% (15% allocation = 122% portfolio contribution)
- MSFT: +915% (15% allocation = 137% portfolio contribution)

**Here's why this works:** Even though TSLA gained 15x, it was only 10% of the portfolio. Trimming reduced it proportionally, but the index funds (60% allocation) provided steady baseline returns that couldn't be "mistakenly trimmed."

**Contrast with individual stock portfolios:** When NVDA gained 280x in our initial testing, it overwhelmed everything. Trimming that position was catastrophically wrong. But in a 60/40 index/stock portfolio, no single winner can dominate enough to make trimming fatal. That's the key.

### 3.2 The Risk-Return Trade-Off

Trimming offered a favorable trade-off for risk-averse investors:

**What you give up:**
- 0.3-1.0% CAGR (21.7% → 21.4% for best trimming strategy)
- $17,967 in final value ($688k → $670k)

**What you get:**
- 5-7% less maximum drawdown (-46% → -40%)
- 4% better Sharpe ratio (0.90 → 0.94)
- Behavioral benefit: Less panic-inducing volatility

**Is this trade worth it?** That's personal. But it's reasonable for investors who:
- Value sleep-at-night portfolio stability over max absolute returns
- Are closer to retirement (less time to recover from drawdowns)
- Have behavioral concerns about holding through 46% drawdowns
- Operate in tax-deferred accounts (no tax drag per trim)

If that's you, the trade makes sense. Otherwise, keep it simple with buy-and-hold.

### 3.3 Trim Threshold Analysis

We tested three thresholds: +50%, +100%, +150%. Here's what we learned: higher thresholds = fewer trims = better performance.

In [ ]:
# Trim frequency analysis
display(Image('visualizations/trim_frequency_analysis.png'))

In [ ]:
# Calculate threshold statistics
threshold_stats = []
for threshold in [50, 100, 150]:
    thresh_strategies = results[results.index.str.contains(f'+{threshold}%')]
    avg_trades = thresh_strategies['num_trades'].mean()
    avg_cagr = thresh_strategies['cagr'].mean()
    avg_sharpe = thresh_strategies['sharpe_ratio'].mean()
    threshold_stats.append({
        'Threshold': f'+{threshold}%',
        'Avg Trades': avg_trades,
        'Avg CAGR': avg_cagr,
        'Avg Sharpe': avg_sharpe
    })

threshold_df = pd.DataFrame(threshold_stats)
print("Trim Threshold Performance Summary")
print("=" * 60)
display(threshold_df)

**What we found:**
- **+50% threshold:** 23 avg trims → 21.16% avg CAGR (overtrimming penalty)
- **+100% threshold:** 14 avg trims → 21.36% avg CAGR (optimal balance)
- **+150% threshold:** 10 avg trims → 21.36% avg CAGR (nearly optimal)

**Sweet spot:** +100-150% thresholds. They trim frequently enough to provide risk reduction but not so frequently that you incur excessive opportunity cost.

**Avoid +50% threshold:** Too aggressive. You're trimming winners way too early and missing significant upside. That 0.2% CAGR penalty proves overtrimming hurts returns without improving risk metrics meaningfully. Don't do it.

### 3.4 CAGR Comparison Across All Strategies

In [ ]:
# CAGR comparison visualization
display(Image('visualizations/cagr_comparison.png'))

## 4. Practical Guidance and Conclusions

### 4.1 When to Use Trimming Strategies

Let me be clear about when trimming works and when it doesn't.

**Trimming works for:**

✅ **Index-heavy portfolios** (60%+ allocation to broad index funds)  
✅ **Risk-averse investors** who value smooth returns over maximum absolute performance  
✅ **Tax-deferred accounts** (IRA, 401k) where trims don't create immediate tax liabilities  
✅ **Behavioral guardrails** - Helps investors avoid panic selling by systematically taking gains  
✅ **Retirees or near-retirees** who can't afford large drawdowns and have shorter recovery horizons  

**Trimming does NOT work for:**

❌ **Concentrated stock portfolios** (less than 10 positions with high-conviction bets)  
❌ **High-conviction growth picks** with 10x-100x potential (you'll trim your winners too early)  
❌ **Taxable accounts** - Tax drag likely eliminates the strategy's viability (see section 4.3)  
❌ **Maximum return seekers** - Buy-and-hold won by 2.6%; if you want absolute max returns, don't trim  
❌ **Stock pickers identifying outliers** - If you can identify NVDA at $0.48, trimming it is insane  

### 4.2 Implementation Guidelines

If trimming fits your profile, here's how to do it right:

**1. Use high thresholds:** +100% or +150%, NOT +50%
   - Lower thresholds overtrim and reduce returns without improving risk metrics
   - Sweet spot: Let winners run to 2x-2.5x before trimming

**2. Reinvest pro-rata:** Maintain proportional exposure across your portfolio
   - Don't concentrate reinvestment into a single position
   - Pro-rata keeps you diversified and exposed to future winners

**3. Don't wait for dips:** Immediate reinvestment beats market timing
   - Even perfect dip execution underperformed by 7%
   - Opportunity cost of sitting in cash exceeds dip discount benefits

**4. Check quarterly, not daily:** Set calendar reminders every 90 days
   - Reduces behavioral temptation to overtrade
   - Prevents emotional decision-making from short-term volatility

**5. Model taxes FIRST:** Run numbers for your specific tax situation
   - Use tax software to estimate actual post-tax returns
   - Compare trimming post-tax CAGR vs buy-and-hold post-tax CAGR
   - Only proceed if trimming still makes sense after taxes

### 4.3 The Tax Problem (Critical Limitation)

**Here's the problem nobody talks about:** Taxes.

Every trim creates a taxable event. In a taxable brokerage account, you pay capital gains tax on every profitable sale:
- **Long-term gains (>1 year):** 15-20% federal tax
- **Short-term gains (<1 year):** 22-37% ordinary income rates

**Impact on our strategy:**
With 14 trims over 10 years, most would qualify as long-term gains. Assuming:
- Average trim proceeds: ~$40,000
- Average gain per trim: ~50-100%
- Rough tax at 15% LTCG: $3,000-$6,000 per trim
- Total tax drag: $42,000-$84,000 over 10 years

This could reduce our 21.4% CAGR to ~19-20% after taxes, potentially eliminating the strategy's edge entirely.

**Bottom line:** This analysis does NOT model taxes. Tax implications are complex and depend on your holding periods, tax bracket, and state taxes. **In taxable accounts, trimming may not be viable once taxes are considered.** In tax-deferred accounts (IRA, 401k), this concern disappears.

**Recommendation:** Consult a tax professional before implementing in taxable accounts. Consider this strategy primarily for tax-advantaged accounts.

### 4.4 Research Limitations

**What this study did NOT test:**

1. **Bear market performance:** 2015-2024 was predominantly bullish. How would trimming perform in 2000-2010 sideways markets or 2008-style crashes?

2. **Different time periods:** Bull markets favor buy-and-hold. Sideways/volatile markets might favor trimming more heavily.

3. **Transaction costs:** Assumed commission-free trading. Even $5 per trade would reduce trimming returns.

4. **Alternative trim percentages:** Only tested 20% position reduction. Would 10%, 30%, or 50% trims perform better?

5. **Dynamic thresholds:** Tested fixed +50/100/150% thresholds. Would volatility-adjusted thresholds improve performance?

6. **Portfolio rebalancing:** Buy-and-hold drifted from initial 60/40 allocation. Would periodic rebalancing change outcomes?

7. **Dividend reinvestment:** Yahoo Finance data includes dividends in price. How would explicit dividend reinvestment affect results?

**Future research directions:**
- Test across multiple market regimes (bull, bear, sideways)
- Model tax implications explicitly for different account types
- Test dynamic threshold adjustment based on VIX or portfolio volatility
- Compare against periodic rebalancing strategies
- Analyze sector rotation with trimming strategies

### 4.5 Final Recommendation

**For most index investors: Stick with buy-and-hold.**

That 2.6% performance edge over 10 years ($688k vs $670k) isn't worth the added complexity, monitoring requirements, and potential tax implications. Simple strategies are often the best strategies. Period.

**Exception: Tax-deferred accounts + risk-averse investors**

If you meet ALL these criteria, trimming is worth considering:
1. Your portfolio is in an IRA, 401k, or other tax-deferred account
2. You have 60%+ allocation to index funds (not concentrated stocks)
3. You value smoother portfolio rides more than maximizing absolute returns
4. You can stick to a disciplined quarterly review process
5. You're comfortable with 21.4% CAGR instead of 21.7% CAGR

**If that's you:** Use Trim@+100% with pro-rata reinvestment. You'll get 97.4% of buy-and-hold returns with materially better risk metrics.

**The key question to ask yourself:**

*Is 6% less maximum drawdown worth 0.3% less CAGR?*

- If yes → Trim at +100% thresholds
- If no → Buy and hold

**The simplest strategy is often the best strategy.** For most investors, that's buy-and-hold.

---

## Appendix: Complete Results Table

In [ ]:
# Display complete results sorted by final value
print("Complete Strategy Performance Results")
print("=" * 120)
complete_results = results.sort_values('final_value', ascending=False)
display(complete_results)

## Reproducibility

**Code location:** `/Users/austinwallace/sandbox/stock_strategies/trim_strat_test/`

**Key files:**
- `run_backtest_index_focus.py` - Backtest engine with all 13 strategies
- `generate_visualizations.py` - Chart generation code
- `manual_data/*.csv` - Historical price data from Yahoo Finance
- `results_index_focus/index_focus_results.csv` - Raw results

**To reproduce:**

```bash
# Run backtest
python run_backtest_index_focus.py

# Generate visualizations
python generate_visualizations.py
```

**Dependencies:**
- Python 3.8+
- pandas, numpy, matplotlib, seaborn
- See `requirements.txt` for complete list

**Data source:** Yahoo Finance historical data (2015-01-01 to 2024-11-05)

**Research date:** November 2024